In [1]:
import torch
from torch import nn
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from datasets import load_dataset
import numpy as np
import psutil
import pandas as pd
from tqdm import tqdm
import time

# 加载数据集和 dcformer 模型

In [2]:
dataset = load_dataset('snli/dataset')
# snli 数据集中含有标签为 -1 的数据, 需去除
dataset = dataset.filter(lambda x: x['label'] != -1)
print(len(dataset['train']), len(dataset['test']), len(dataset['validation']))

549367 9824 9842


In [4]:
dataset['train'][0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1}

In [3]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
modelpath = os.getcwd() + '/dcformer28b'
print("模型所在路径位置: ", modelpath)
sys.path.append(modelpath)

模型所在路径位置:  /root/autodl-tmp/dcformer/dcformer28b


In [4]:
tokenizer = AutoTokenizer.from_pretrained("dcformer28b", padding=True, truncation=True, max_length=500)
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/dcformer/dcformer28b", trust_remote_code=True)

device = torch.device('cuda')
# device = torch.device('cpu')
MAX_BATCH_SIZE = 1
MAX_SEQ_LENGTH = 200
NUM_TOKENS_TO_GENERATE = 100
COMPILE = True

_ = model.to(device=device, dtype=torch.float16)
with torch.device(device):
    model.setup_caches(max_batch_size=MAX_BATCH_SIZE, max_seq_length=MAX_SEQ_LENGTH, set_kv_cache=False)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


# 获取 premise, hypothesis 的输出向量

## 训练集

### 两层注意力块输出

In [6]:
# premise + hypothesis
seq_length = 128
features_out = []

def layer_hook(module, fea_in, output):
    # output 尺寸为: (batch * sen_len * embedding)
    # 对第 1 维度求均值, features_out 存储的 array 尺寸: (batch * embedding)
    features_out.append(np.mean(output.data.cpu().numpy(), axis=1))
    return None


for i, x in enumerate(dataset['train']):
    # print(f"正在读取第 {i + 1} 条数据, 当前 cpu 占用率: {psutil.cpu_percent()}%")
    print(f"正在读取第 {i + 1} 条数据")
    h, p = x['premise'], x['hypothesis']
    x_token_ids = tokenizer.encode(h + p, return_tensors='pt')
    x_token_ids = tokenizer.encode(text, return_tensors='pt')
    if x_token_ids.shape[1] > seq_length:
        x_token_ids = x_token_ids[:, :seq_length]
    with torch.no_grad():
        h = model.layers[1].attention_norm.register_forward_hook(hook=layer_hook)
        _ = model(x_token_ids.to(device))
        del _
        h.remove()
        del h
    if i % 1000 == 0:
        print(f"已处理: {len(features_out)} 条数据")
        for _ in range(500):
            torch.cuda.empty_cache()

正在读取第 1 条数据
已处理: 1 条数据
正在读取第 2 条数据
正在读取第 3 条数据
正在读取第 4 条数据
正在读取第 5 条数据
正在读取第 6 条数据
正在读取第 7 条数据
正在读取第 8 条数据
正在读取第 9 条数据
正在读取第 10 条数据
正在读取第 11 条数据
正在读取第 12 条数据
正在读取第 13 条数据
正在读取第 14 条数据
正在读取第 15 条数据
正在读取第 16 条数据
正在读取第 17 条数据
正在读取第 18 条数据
正在读取第 19 条数据
正在读取第 20 条数据
正在读取第 21 条数据
正在读取第 22 条数据
正在读取第 23 条数据
正在读取第 24 条数据
正在读取第 25 条数据
正在读取第 26 条数据
正在读取第 27 条数据
正在读取第 28 条数据
正在读取第 29 条数据
正在读取第 30 条数据
正在读取第 31 条数据
正在读取第 32 条数据
正在读取第 33 条数据
正在读取第 34 条数据
正在读取第 35 条数据
正在读取第 36 条数据
正在读取第 37 条数据
正在读取第 38 条数据
正在读取第 39 条数据
正在读取第 40 条数据
正在读取第 41 条数据
正在读取第 42 条数据
正在读取第 43 条数据
正在读取第 44 条数据
正在读取第 45 条数据
正在读取第 46 条数据
正在读取第 47 条数据
正在读取第 48 条数据
正在读取第 49 条数据
正在读取第 50 条数据
正在读取第 51 条数据
正在读取第 52 条数据
正在读取第 53 条数据
正在读取第 54 条数据
正在读取第 55 条数据
正在读取第 56 条数据
正在读取第 57 条数据
正在读取第 58 条数据
正在读取第 59 条数据
正在读取第 60 条数据
正在读取第 61 条数据
正在读取第 62 条数据
正在读取第 63 条数据
正在读取第 64 条数据
正在读取第 65 条数据
正在读取第 66 条数据
正在读取第 67 条数据
正在读取第 68 条数据
正在读取第 69 条数据
正在读取第 70 条数据
正在读取第 71 条数据
正在读取第 72 条数据
正在读取第 73 条数据
正在读取第 74 条数据
正在读取第 75 条数据
正在读取第 76 条数据
正在读取第 77 条

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [9]:
print("已处理 %d 条数据。" % len(features_out))

已处理 59002 条数据。


In [16]:
train_premise_embedding = np.array(list(np.squeeze(arr) for arr in features_out))
print(f"保存数组尺寸: {train_premise_embedding.shape}")
np.save('snli/2layer_train_embedding.npy', train_premise_embedding)

保存数组尺寸: (59002, 2560)


# 测试集

In [ ]:
seq_length = 128
features_out = []

def layer_hook(module, fea_in, output):
    # output 尺寸为: (batch * sen_len * embedding)
    # 对第 1 维度求均值, features_out 存储的 array 尺寸: (batch * embedding)
    features_out.append(np.mean(output.cpu().numpy(), axis=1))
    return None


for i, x in enumerate(dataset['test']):
    if i == 1351:
        continue
    print(f"正在读取第 {i + 1} 条数据, 当前 cpu 占用率: {psutil.cpu_percent()}%")
    # print(f"正在读取第 {i + 1} 条数据")
    h, p = x['premise'], x['hypothesis']
    x_token_ids = tokenizer.encode(h + p, return_tensors='pt')
    if x_token_ids.shape[1] > seq_length:
        x_token_ids = x_token_ids[:, :seq_length]
    with torch.no_grad():
        h = model.layers[1].attention_norm.register_forward_hook(hook=layer_hook)
        _ = model(x_token_ids.to(device))
        del _
        h.remove()
        del h
    if i % 1000 == 0:
        print(f"已处理: {len(features_out)} 条数据")
        for _ in range(500):
            torch.cuda.empty_cache()

In [ ]:
test_premise_embedding = np.array(list(np.squeeze(arr) for arr in features_out))
print(f"保存数组尺寸: {test_premise_embedding.shape}")
np.save('snli/2layer_test_embedding.npy', test_premise_embedding)

# 加载数据集

In [ ]:
dataset = load_dataset('snli/dataset')

train_array = np.load('snli/2layer_train_embedding.npy')
print(f"训练数据集输入尺寸: {train_array.shape}")
y_train = torch.tensor(dataset['train']['label'])
print(f"训练数据集标签尺寸: {y_train.shape}")

test_array = np.load('snli/test_embedding.npy')
y_test = torch.tensor(dataset['test']['label'])

In [ ]:
from torch.utils.data import Dataset
from datasets import load_dataset


class SnliDataset(Dataset):
    def __init__(self, x_array, y):
        self.x_data = torch.tensor(x_array, dtype=torch.float32)
        self.y_data = y


    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    
    def __len__(self):
        return len(self.x_data)

In [ ]:
train_set = ImdbDataset(train_array, y_train)
test_set = ImdbDataset(test_array, y_test)

In [ ]:
batch_size = 512
# 生成可迭代数据集
train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_set, batch_size, shuffle=False)
print(f'训练集大小: {len(train_iter)}, 测试集大小: {len(test_iter)}')

# 模型架构

In [ ]:
class NetForSnli(nn.Module):
    def __init__(self, num_class, embedding_size=2560):
        super(NetForImdb, self).__init__()
        self.layer = nn.Sequential(
                                nn.Linear(embedding_size * 2, embedding_size),
                                nn.ReLU(),
                                nn.Linear(embedding_size, embedding_size),
                                nn.ReLU(),
                                nn.Linear(embedding_size, num_class))


    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=-1)
        return self.layer(x)

# 实例化和初始化模型参数

In [ ]:
net = NetForSnli(num_class=3)
total_trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

for m in net.modules():
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)

In [ ]:
# 定义评估模型准确率和 F1 指标的函数
from sklearn.metrics import accuracy_score, f1_score
# f1_score(labels, predicitons, average='micro')
def evaluate_accuracy_f1(net, data_iter, device=None):
    net.eval()  # Set the model to evaluation mode
    if not device:
        device = next(iter(net.parameters())).device

    y_pred, y_true = torch.tensor([], device=device), torch.tensor([], device=device)
    with torch.no_grad():
        for x1, x2, y in data_iter:
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)
            y_pred = torch.cat([y_pred, net(x1, x2).argmax(dim=-1)])
            y_true = torch.cat([y_true, y])
    return accuracy_score(y_pred.cpu(), y_true.cpu()), f1_score(y_pred.cpu(), y_true.cpu())

# 模型训练和评估

In [ ]:
import time
from tqdm import tqdm

num_batches = len(train_iter)
losses, train_accuracy, test_accuracy = [], [], []
train_f1, test_f1 = [], []
total_start = time.time()
for epoch in range(num_epochs):
    train_loss = 0
    start_time = time.perf_counter()
    net.train()
    for index, data in tqdm(enumerate(train_iter), total=num_batches, leave=True):
        x1, x2, y = data
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        l = loss(net(x1, x2), y)
        trainer.zero_grad()
        l.sum().backward()
        trainer.step()
        train_loss += l.sum().cpu().detach().numpy() / batch_size
        # train_acc += d2l.accuracy(net(X, valid_len), y) / batch_size
    epochloss = train_loss / num_batches
    epochtrainacc, epochtrainf1 = evaluate_accuracy_f1(net, train_iter)
    epochtestacc, epochtestf1 = evaluate_accuracy_f1(net, test_iter)
    end_time = time.perf_counter()
    print(f'epoch {epoch + 1}, loss {epochloss:.3f}, train_acc {epochtrainacc:.3f}, '
        f'train_f1 {epochtrainf1:.3f}, test_acc {epochtestacc:.3f}, '
        f'test_f1 {epochtestf1:.3f}, taking {round(end_time - start_time)} seconds')
    losses.append(epochloss)
    train_accuracy.append(epochtrainacc)
    train_f1.append(epochtrainf1)
    test_accuracy.append(epochtestacc)
    test_f1.append(epochtestf1)
total_timecost = round(time.time() - total_start)
print('Training is finished.')
print(f'训练损失:{losses[-1]:.3f}, 训练集准确率:{train_accuracy[-1]:.3f}, '
      f'训练集f1:{train_f1[-1]:.3f}, 测试集准确率:{test_accuracy[-1]:.3f}, '
     f'测试集f1:{test_f1[-1]:.3f}, 在设备{device}上训练{num_epochs}周期, 耗时{total_timecost} s')